# lab6 决策树

> 姓名：王茂增\
> 学号：2113972\
> 专业：计算机科学与技术\
> 代码：https://github.com/mzwangg/MachineLearning

## 实验要求

### 基本要求

a)	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；

b)	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；

### 中级要求

a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；

b)	对测试集Watermelon-test2进行预测，输出分类精度；

### 高级要求

使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

## 基本要求: ID3决策树

### 读取数据

为了在高级要求时实现剪枝，我将train.csv数据集通过**分层采样**进一步划分为了**训练集**和**验证集**，具体见代码及注释。

In [1986]:
import numpy as np
import pandas as pd

# 对train.csv进行分层采样
def load_train_data(file_path, seed, validation_ratio=0.3):
    # 设置随机数种子， 以保证可复现性
    np.random.seed(seed)
    # 读取数据，获取列标签
    data = np.array(pd.read_csv(file_path, encoding="GBK"))[:,1:]
    labels = data[:, -1]
    unique_labels = np.unique(labels)

    # 初始化训练集和验证集
    train_set = []
    validation_set = []

    # 对每个唯一标签进行分层采样
    for label in unique_labels:
        # 获取属于当前标签的所有样本的索引，随机打乱，计算划分点
        indices = np.where(labels == label)[0]
        np.random.shuffle(indices)
        split_point = int(len(indices) * (1 - validation_ratio))

        # 划分数据集
        train_indices = indices[:split_point]
        validation_indices = indices[split_point:]

        # 将数据添加到训练集和验证集
        train_set.append(data[train_indices])
        validation_set.append(data[validation_indices])

    # 将分层采样后的数据集合并为最终的训练集和验证集
    train_set = np.concatenate(train_set, axis=0)
    validation_set = np.concatenate(validation_set, axis=0)

    return train_set, validation_set

def load_test_data(file_path):
    data = np.array(pd.read_csv(file_path, encoding="GBK"))[:,1:]
    return data

def load_feature(file_path):
    data = np.genfromtxt(file_path, delimiter=',', dtype=str, max_rows=1, encoding='GBK')
    return data[1:]

# 加载数据
train_data1, valid_data1 = load_train_data("Watermelon-train1.csv", 10)
test_data1 = load_test_data("Watermelon-test1.csv")
feature1 = load_feature("Watermelon-train1.csv")

train_data2, valid_data2 = load_train_data("Watermelon-train2.csv", 9)
test_data2 = load_test_data("Watermelon-test2.csv")
feature2 = load_feature("Watermelon-train2.csv")

### 信息增益：

1. **信息熵（Entropy）公式**：

    $ \text{Entropy}(D) = - \sum_{i=1}^{k} p_i \cdot \log_2(p_i) $

    其中，$k$ 是类别数目，$p_i$ 是数据集中属于类别 $i$ 的样本占比。

2. **信息增益公式**:

    $ \text{Gain}(D, A) = \text{Entropy}(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \cdot \text{Entropy}(D_v) $

    其中，$\text{Gain}(D, A)$ 是信息增益，$\text{Entropy}(D)$ 是数据集 $D$ 的信息熵，$\text{Values}(A)$ 是特征 $A$ 的取值集合，$D_v$ 是特征 $A$ 取值为 $v$ 时的子集。

In [1987]:
# 计算信息熵
def entropy(labels):
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    return -np.sum(probabilities * np.log2(probabilities))

# 计算信息增益
def information_gain(data, feature_index):
    features = data[:, feature_index]
    unique_values, counts = np.unique(features, return_counts=True)
    w_entropys = np.sum([(counts[i] / len(features)) * 
                         entropy(data[data[:, feature_index] 
                                      == unique_values[i]][:, -1]) 
                         for i in range(len(unique_values))])
    return entropy(data[:, -1]) - w_entropys



### ID3决策树

**ID3 (Iterative Dichotomiser 3)** 是一种用于构建决策树的经典算法，最早由Ross Quinlan于1986年提出。它是基于信息熵的一种自顶向下的递归分割算法，旨在通过选择最佳的特征进行节点分割，使得每个分割后的子集尽可能纯净。

### ID3算法步骤：

1. **选择最佳特征：** 通过计算每个特征的信息增益（信息熵的减少量）来选择最佳的特征作为当前节点的分割特征。信息增益表示使用该特征进行分割后数据集的不确定性减少程度。

2. **为节点分配标签：** 如果当前节点的数据集所有样本都属于同一类别，则将该节点标记为叶子节点，并使用该类别标签。

3. **递归：** 对于每个特征值，将数据集划分为子集，并对每个子集应用上述步骤，递归构建决策树。

在递归构建决策树节点的过程中，将此时的训练集标签也加入节点，以便进行剪枝。

ID3算法的主要优点是简单易懂，生成的决策树具有可解释性。然而，它对于处理连续特征和处理缺失值的能力相对较弱，这些问题后来由C4.5和CART等算法得到改进。

In [1988]:
def id3_decision_tree(data, feature_indices):
    # 如果没有特征可用，或已经全部划分完成则返回数据集中实例数最多的类别
    unique_labels, counts = np.unique(data[:, -1], return_counts=True)
    if len(unique_labels) == 1 or len(feature_indices) == 0:
        return unique_labels[np.argmax(counts)]
    
    # 选择最佳划分特征，并更新feature_indices
    best_feature_index = np.argmax([information_gain(data, i) for i in feature_indices])
    feature_indices = np.delete(feature_indices, best_feature_index)
    
    # 根据选取的特征在在子树上递归继续构建子树
    # 将此时的训练集标签也加入节点，以便进行剪枝
    tree = (best_feature_index, [], data[:, -1])
    unique_values = np.unique(data[:, best_feature_index])
    for value in unique_values:
        subset = data[data[:, best_feature_index] == value]
        subtree = id3_decision_tree(subset, feature_indices)
        tree[1].append((value, subtree))
    
    return tree

### 预测及打印

In [1989]:
# 递归进行预测
def id3_predict(tree, sample):
    # 当预测到类别时，直接返回预测的类别
    if not isinstance(tree, tuple):
        return tree
    
    # 根据建立的决策树递归预测
    feature, subtrees, _ = tree
    for subtree in subtrees:
        if subtree[0] == sample[feature]:
            return id3_predict(subtree[1], sample)

# 打印决策树
def print_id3_tree(node, features, indent=""):
    if isinstance(node, tuple):
        feature_index, branches, _ = node
        feature_name = features[feature_index]
        print(f"{indent}{feature_name}")
        for value, subtree in branches:
            print(f"{indent}  |---{feature_name} == {value}")
            print_id3_tree(subtree, features, indent + "  |   ")
    else:
        print(f"{indent}  └──类别: {node}")

### 建树及预测

In [1990]:
# 构建ID3决策树
feature_indices1 = np.arange(train_data1.shape[1] - 1)
id3_tree = id3_decision_tree(train_data1, feature_indices1)

# 预测测试集并计算分类精度
test_true1 = test_data1[:, -1]
id3_predict(id3_tree, test_data1[:, :-1][0])
test_pred1 = np.array([id3_predict(id3_tree, sample) for sample in test_data1[:, :-1]])
acc1 = np.sum(test_pred1 == test_true1) / len(test_true1)

# 输出预测精度
print("test_true:", test_true1)
print("test_pred:", test_pred1)
print("ID3决策树分类精度:", acc1)

test_true: ['是' '是' '是' '是' '是' '否' '否' '否' '否' '否']
test_pred: ['是' '是' '是' '否' '是' '是' '是' '否' '否' '否']
ID3决策树分类精度: 0.7


## 中级要求：CART决策树

**CART决策树（Classification and Regression Trees）** 是由Leo Breiman于1984年提出的一种用于分类和回归的树形模型。CART决策树的主要特点是可以处理离散和连续型的特征，并且可以应用于分类和回归任务。

### CART决策树的优点和缺点：

**优点：**
- 可以处理混合类型的特征（离散和连续）。
- 生成的树具有很好的解释性。
- 对于较小的数据集和较小的树，具有较高的计算效率。

**缺点：**
- 对于高维稀疏数据或大规模数据，可能性能不如其他模型。
- 生成的树可能过拟合，需要剪枝等策略进行调整。

CART决策树在许多实际应用中都表现良好，特别适用于二分类和回归问题。

### 基尼系数

$ \text{Gini}(D) = 1 - \sum_{i=1}^{k} p_i^2 $

其中，$k$ 是类别数目，$p_i$ 是数据集 $D$ 中属于类别 $i$ 的样本占比。

In [1991]:
# 根据相应公式计算基尼系数
def calculate_gini(labels):
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    return 1 - np.sum(probabilities**2) 

### CART决策树的构建过程：

1. **特征选择：** CART使用**基尼系数**作为特征选择的准则。对于一个数据集，对于每个特征的每个可能的切分点，计算切分后的子集的基尼系数。选择具有最小基尼不纯度的特征和切分点。

2. **递归分裂：** 将数据集根据选择的特征和切分点进行分裂，得到子集。对每个子集重复上述过程，直到满足停止条件（如树的深度达到预定值、节点中的样本数小于某个阈值等）。

3. **生成叶子节点：** 当满足停止条件时，为叶子节点分配一个输出值。对于分类任务，通常选择叶子节点中样本最多的类别作为输出值；对于回归任务，通常选择叶子节点中样本的平均值作为输出值。

In [1992]:
# 将数据data在feature_index属性上按照threshold划分左右子树
def split_dataset(data, feature_index, threshold):
    if isinstance(threshold, str): # 处理字符串
        left_mask = data[:, feature_index] == threshold
        right_mask = ~left_mask
    else: # 处理连续值
        left_mask = data[:, feature_index] <= threshold
        right_mask = ~left_mask
    return data[left_mask], data[right_mask] # 返回左子树和右子树

# 寻找最优划分
def find_best_split(data):
    best_gini = float('inf')
    best_split = None
    
    for i in range(data.shape[1] - 1):# 遍历每个属性
        unique_values = np.unique(data[:, i])
        for value in unique_values:# 遍历属性的每个值
            left_data, right_data = split_dataset(data, i, value) # 进行划分

            # 计算左右子树的基尼系数，并进一步计算整体的基尼系数
            gini_left = calculate_gini(left_data[:, -1])
            gini_right = calculate_gini(right_data[:, -1])
            gini = (len(left_data) * gini_left + len(right_data) * gini_right) / len(data)

            # 选择具有最小基尼系数的划分
            if gini < best_gini:
                best_gini = gini
                best_split = (i, value)

    return best_split

# 构建CART决策树
def cart_decision_tree(data, max_depth=float('inf')):
    # 当达到最大深度或者已经全部划分完成，返回数量最多的类别作为预测类别
    unique_labels, counts = np.unique(data[:, -1], return_counts=True)
    if max_depth == 0 or len(unique_labels) == 1:
        return unique_labels[np.argmax(counts)]
    
    # 寻找的当前数据的最优划分，并得到最优划分的左右子树
    best_split = find_best_split(data)
    feature_index, threshold = best_split
    left_data, right_data = split_dataset(data, feature_index, threshold)

    # 对左右子树进一步进行划分
    left_subtree = cart_decision_tree(left_data, max_depth - 1)
    right_subtree = cart_decision_tree(right_data, max_depth - 1)
    return [feature_index, threshold, left_subtree, right_subtree, data[:, -1]]

### 预测及打印

In [1993]:
def cart_predict(tree, sample):
    # 如果到达了叶子结点，则tree只是一个类别。而不是tuple构成的树
    if not isinstance(tree, list):
        return tree

    feature_index, threshold, left_subtree, right_subtree, _ = tree
    # 根据划分属性的类别计算到达左右子树的数据序号
    if isinstance(threshold, str):  # 如果当前划分属性是字符串
        condition = sample[feature_index] == threshold
    else:  # 如果当前划分属性是连续值
        condition = sample[feature_index] <= threshold

    # 选择左右子树中的一个进行递归预测
    if condition:
        return cart_predict(left_subtree, sample)
    else:
        return cart_predict(right_subtree, sample)

def print_cart_tree(node, features, indent=""):
    if len(node) == 1:
        # 叶子节点，直接打印类别
        print(f"{indent}Class: {node[0]}")
        return
    
    feature_index, threshold, left_subtree, right_subtree, _ = node
    feature_name = features[feature_index]

    # 打印当前节点判断条件
    print(f"{indent}{feature_name} <= {threshold}")
    # 递归打印左子树
    print_cart_tree(left_subtree, features, indent + "  |   ")
    # 打印当前节点的判断条件
    print(f"{indent}  └── {feature_name} > {threshold}")
    # 递归打印右子树
    print_cart_tree(right_subtree, features, indent + "      ")

### 建树及预测

In [1994]:
# 使用CART算法建立决策树
cart_tree = cart_decision_tree(train_data2)

# 计算预测准确率
test_true2 = test_data2[:, -1]
test_pred2 = np.array([cart_predict(cart_tree, sample) for sample in test_data2[:, :-1]])
acc2 = np.sum(test_pred2 == test_true2) / len(test_true2)

# 输出预测精度
print("test_true:", test_true2)
print("test_pred:", test_pred2)
print("ID3决策树分类精度:", acc2)

test_true: ['是' '是' '是' '否' '否']
test_pred: ['是' '否' '是' '是' '否']
ID3决策树分类精度: 0.6


## 高级要求：剪枝

后剪枝（post-pruning）是决策树构建过程的一个阶段，它发生在决策树已经建立完成后。后剪枝的目标是通过剪枝一些分支（子树）来提高决策树的泛化性能。以下是决策树后剪枝的一般步骤：

1. **划分数据集：** 将原始数据集划分为训练集、验证集、测试集。训练集用于构建决策树，而验证集用于评估剪枝后的性能。

2. **构建完整的决策树：** 首先，通过训练数据构建完整的决策树。这个决策树可能在训练数据上表现得很好，但在新数据上可能存在过拟合的问题。

3. **后序遍历决策树：** 从决策树的底部（叶子节点）开始，进行后序遍历。在遍历过程中，每个节点分别作为根节点进行以下操作。

4. **剪枝尝试：** 用当前节点的多数类别替换子树，并在验证集上评估性能。如果剪枝后的性能不下降（甚至有所提升），则执行剪枝操作，将子树替换为当前节点的多数类别。否则，保留原来的子树。

5. **递归操作：** 递归地对每个父节点进行相同的操作，直到根节点。

这样，通过在验证集上评估剪枝后的性能，决策树的复杂部分可以被替换为更简单的结构，从而提高了决策树的泛化性能。在实际应用中，我们可以调整验证集的划分比例、剪枝时机等参数来优化后剪枝的效果。

### ID3决策树剪枝

In [1995]:
# 评估id3决策树在数据集data上的精确度
def cal_id3_acc(data):
    true = data[:, -1]
    pred = np.array([id3_predict(id3_tree, sample) for sample in data[:, :-1]])
    acc = np.sum(pred == true) / len(true)
    return acc if not np.isnan(acc) else 0

# id3决策树后剪枝
def id3_post_pruning(parent, index, tree):
    if not isinstance(tree, tuple):
        return

    # 遍历子树，递归进行后剪枝
    subtrees = tree[1]
    index = tree[0]
    for i in range(len(subtrees)):
        subtree = subtrees[i]
        id3_post_pruning(subtrees, i, subtree[1])
    
    if parent is None:
        return
        
    # 计算当前节点的正确率
    original_acc = cal_id3_acc(valid_data1)
        
    # 将待剪枝节点替换为训练集在该子树中最多的那个类型
    old_tree = parent[index]
    labels = tree[2]
    unique_elements, counts = np.unique(labels, return_counts=True)
    index_of_max_count = np.argmax(counts)
    parent[index] = (old_tree[0], unique_elements[index_of_max_count])

    # 计算剪枝之后在验证集的正确率
    after_acc = cal_id3_acc(valid_data1)

    # 如果合并后在验证集中正确率没有增加，则恢复子树
    if after_acc <= original_acc:
        parent[index] = old_tree

In [1996]:
print("剪枝前：\n")
print_id3_tree(id3_tree, feature1)
print("\n正确率:", cal_id3_acc(test_data1))

剪枝前：

纹理
  |---纹理 == 模糊
  |     └──类别: 否
  |---纹理 == 清晰
  |     └──类别: 是
  |---纹理 == 稍糊
  |   敲声
  |     |---敲声 == 沉闷
  |     |     └──类别: 否
  |     |---敲声 == 浊响
  |     |     └──类别: 是

正确率: 0.7


In [1997]:
id3_post_pruning(None, -1, id3_tree)
print("剪枝后：\n")
print_id3_tree(id3_tree, feature1)
print("\n正确率:", cal_id3_acc(test_data1))

剪枝后：

纹理
  |---纹理 == 模糊
  |     └──类别: 否
  |---纹理 == 清晰
  |     └──类别: 是
  |---纹理 == 稍糊
  |     └──类别: 否

正确率: 0.8


有上述结果可知，经过剪枝之后，当纹理等于稍糊时，决策树由原来的进一步根据敲声判断变为了直接返回否，这不仅减少了决策树的规模，也将决策树的准确率由0.7提高到了0.8。

### CART决策树剪枝

In [1998]:
# 评估cart决策树在数据集data上的精确度
def cal_cart_acc(data):
    true = data[:, -1]
    pred = np.array([cart_predict(cart_tree, sample) for sample in data[:, :-1]])
    acc = np.sum(pred == true) / len(true)
    return acc if not np.isnan(acc) else 0

# cart决策树后剪枝
def cart_post_pruning(parent, index, tree):
    if not isinstance(tree, list):
        return

    # 对左右子树递归进行后剪枝
    cart_post_pruning(tree, 2, tree[2])
    cart_post_pruning(tree, 3, tree[3])
    
    if parent is None:
        return
        
    # 计算当前节点的正确率
    original_acc = cal_cart_acc(valid_data2)

    # 将待剪枝节点替换为训练集在该子树中最多的那个类型
    old_tree = parent[index]
    labels = tree[4]
    unique_elements, counts = np.unique(labels, return_counts=True)
    index_of_max_count = np.argmax(counts)
    parent[index] = unique_elements[index_of_max_count]

     # 计算剪枝之后在验证集的正确率
    after_acc = cal_cart_acc(valid_data2)

    # 如果合并后在验证集中正确率没有增加，则恢复子树
    if after_acc <= original_acc:
        parent[index] = old_tree

In [1999]:
print("剪枝前：\n")
print_cart_tree(cart_tree, feature2)
print("\n正确率:", cal_cart_acc(test_data2))

剪枝前：

纹理 <= 清晰
  |   根蒂 <= 硬挺
  |     |   Class: 否
  |     └── 根蒂 > 硬挺
  |         Class: 是
  └── 纹理 > 清晰
      色泽 <= 乌黑
        |   Class: 是
        └── 色泽 > 乌黑
            Class: 否

正确率: 0.6


In [2000]:
cart_post_pruning(None, -1, cart_tree)
print("剪枝后：\n")
print_cart_tree(cart_tree, feature2)
print("\n正确率:", cal_cart_acc(test_data2))

剪枝后：

纹理 <= 清晰
  |   根蒂 <= 硬挺
  |     |   Class: 否
  |     └── 根蒂 > 硬挺
  |         Class: 是
  └── 纹理 > 清晰
      Class: 否

正确率: 0.8


有上述结果可知，经过剪枝之后，当纹理不等于清晰时，决策树由原来的进一步根据色泽判断变为了直接返回否，这不仅减少了决策树的规模，也将决策树的准确率由0.6提高到了0.8。